In [111]:
using JuMP, HiGHS

In [112]:
oferta = [400, 200, 200, 10]  # con nodo ficticio
demanda = [100, 200, 150, 160, 200]
distancia = [
    100 150 200 140 35;
    50  70  60  65 80;
    40  90 100 150 130;
    1000 1000 1000 1000 1000
]

costo_por_milla = 25  # Costo por milla de cada camión
capacidad_camion = 18  # Máximo de autos por camión


18

In [113]:
modelo = Model(HiGHS.Optimizer)

A JuMP Model
├ solver: HiGHS
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 0
├ num_constraints: 0
└ Names registered in the model: none

In [114]:
@variable(modelo, x[1:4, 1:5] >= 0) # No negativos

4×5 Matrix{VariableRef}:
 x[1,1]  x[1,2]  x[1,3]  x[1,4]  x[1,5]
 x[2,1]  x[2,2]  x[2,3]  x[2,4]  x[2,5]
 x[3,1]  x[3,2]  x[3,3]  x[3,4]  x[3,5]
 x[4,1]  x[4,2]  x[4,3]  x[4,4]  x[4,5]

In [115]:
# Variables auxiliares: número de camiones usados (enteras)
@variable(modelo, y[1:4, 1:5] >= 0, Int)

4×5 Matrix{VariableRef}:
 y[1,1]  y[1,2]  y[1,3]  y[1,4]  y[1,5]
 y[2,1]  y[2,2]  y[2,3]  y[2,4]  y[2,5]
 y[3,1]  y[3,2]  y[3,3]  y[3,4]  y[3,5]
 y[4,1]  y[4,2]  y[4,3]  y[4,4]  y[4,5]

In [116]:
# Función objetivo: minimizar el costo total de transporte
@objective(modelo, Min, sum(
    costo_por_milla * distancia[i, j] * y[i, j] 
    for i in 1:4, j in 1:5
))

2500 y[1,1] + 3750 y[1,2] + 5000 y[1,3] + 3500 y[1,4] + 875 y[1,5] + 1250 y[2,1] + 1750 y[2,2] + 1500 y[2,3] + 1625 y[2,4] + 2000 y[2,5] + 1000 y[3,1] + 2250 y[3,2] + 2500 y[3,3] + 3750 y[3,4] + 3250 y[3,5] + 25000 y[4,1] + 25000 y[4,2] + 25000 y[4,3] + 25000 y[4,4] + 25000 y[4,5]

In [117]:
# Restricciones de oferta
for i in 1:4
    @constraint(modelo, sum(x[i, j] for j in 1:5) <= oferta[i])
end

In [118]:
# Restricciones de demanda
for j in 1:5
    @constraint(modelo, sum(x[i, j] for i in 1:4) == demanda[j])
end

In [119]:
# Restricción: número de autos por camión
for i in 1:4, j in 1:5
    @constraint(modelo, x[i, j] <= capacidad_camion * y[i, j])
end

In [120]:
optimize!(modelo)

Running HiGHS 1.7.2 (git hash: 5ce7a2753): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 2e+01]
  Cost   [9e+02, 2e+04]
  Bound  [0e+00, 0e+00]
  RHS    [1e+01, 4e+02]
Presolving model
29 rows, 40 cols, 80 nonzeros  0s
29 rows, 40 cols, 80 nonzeros  0s
Objective function is integral with scale 0.008

Solving MIP model with:
   29 rows
   40 cols (0 binary, 20 integer, 0 implied int., 20 continuous)
   80 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               inf                  inf        0      0      0         0     0.0s
 S       0       0         0   0.00%   0               115250           100.00%        0      0      0         0     0.0s
         0       0         0   0.00%   109583.333333 

In [121]:
if termination_status(modelo) == MOI.OPTIMAL
    println("Distribución óptima:")
    for i in 1:4, j in 1:5
        autos = value(x[i, j])
        camiones = value(y[i, j])
        println("Enviar $autos autos del Centro $i al Concesionario $j (Usar $camiones camiones)")
    end
    println("Costo total: \$", objective_value(modelo))
else
    println("No se encontró solución óptima.")
end

Distribución óptima:
Enviar 40.0 autos del Centro 1 al Concesionario 1 (Usar 3.0 camiones)
Enviar 0.0 autos del Centro 1 al Concesionario 2 (Usar -0.0 camiones)
Enviar 0.0 autos del Centro 1 al Concesionario 3 (Usar -0.0 camiones)
Enviar 160.0 autos del Centro 1 al Concesionario 4 (Usar 9.0 camiones)
Enviar 200.0 autos del Centro 1 al Concesionario 5 (Usar 12.0 camiones)
Enviar 0.0 autos del Centro 2 al Concesionario 1 (Usar -0.0 camiones)
Enviar 50.0 autos del Centro 2 al Concesionario 2 (Usar 3.0 camiones)
Enviar 150.0 autos del Centro 2 al Concesionario 3 (Usar 9.0 camiones)
Enviar -0.0 autos del Centro 2 al Concesionario 4 (Usar -0.0 camiones)
Enviar 0.0 autos del Centro 2 al Concesionario 5 (Usar -0.0 camiones)
Enviar 60.0 autos del Centro 3 al Concesionario 1 (Usar 4.0 camiones)
Enviar 140.0 autos del Centro 3 al Concesionario 2 (Usar 8.0 camiones)
Enviar 0.0 autos del Centro 3 al Concesionario 3 (Usar -0.0 camiones)
Enviar 0.0 autos del Centro 3 al Concesionario 4 (Usar -0.0 cam